## Associated Doc: 

https://docs.google.com/spreadsheets/d/1LQOaf9-Bu6N8cjxAX6gsMfgi3-NMrT9njdZplBfWQ0A/edit?gid=0#gid=0

## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Data Read in

In [2]:
df = pd.read_csv('South Florida sites hitting the market  - Sheet1.csv')

In [3]:
df.columns = df.columns.str.strip()

In [4]:
import math
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from html import escape

# -----------------------------
# 1) Helpers
# -----------------------------
def parse_latlon(val):
    """Return (lat, lon) as floats or (None, None) if unparseable/NaN."""
    if pd.isna(val):
        return (None, None)
    # Accept strings like "lat, lon" or tuples
    if isinstance(val, str):
        parts = [p.strip() for p in val.split(",")]
        if len(parts) != 2:
            return (None, None)
        try:
            lat = float(parts[0])
            lon = float(parts[1])
            # sanity check
            if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                return (None, None)
            return (lat, lon)
        except ValueError:
            return (None, None)
    if isinstance(val, (list, tuple)) and len(val) == 2:
        lat, lon = val
        try:
            lat = float(lat); lon = float(lon)
            if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                return (None, None)
            return (lat, lon)
        except ValueError:
            return (None, None)
    return (None, None)

def make_popup_html(row):
    prop   = row.get("Property (name or address)", "")
    cap    = row.get("Caption to use in map", "")
    link   = row.get("Story link", "")
    # Escape user text to avoid HTML injection
    prop_e = escape(str(prop)) if not pd.isna(prop) else ""
    cap_e  = escape(str(cap))  if not pd.isna(cap)  else ""
    link_e = str(link) if not pd.isna(link) else ""

    link_html = f'<div><a href="{escape(link_e)}" target="_blank" rel="noopener">Read story ↗</a></div>' if link_e else ""
    return f"""
    <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; max-width: 320px;">
      <div style="font-weight:600; margin-bottom:4px;">{prop_e}</div>
      <div style="margin-bottom:6px; line-height:1.3;">{cap_e}</div>
      {link_html}
    </div>
    """

# -----------------------------
# 2) Prepare coordinates
# -----------------------------
latlons = df["Google coordinates"].apply(parse_latlon)
df["_lat"] = latlons.apply(lambda t: t[0])
df["_lon"] = latlons.apply(lambda t: t[1])

valid = df.dropna(subset=["_lat", "_lon"])
skipped = len(df) - len(valid)

# -----------------------------
# 3) Make the map
# -----------------------------
if len(valid):
    center = [valid["_lat"].mean(), valid["_lon"].mean()]
    zoom_start = 12
else:
    # Fallback to continental US if nothing valid
    center = [39.5, -98.35]
    zoom_start = 4

m = folium.Map(location=center, zoom_start=zoom_start, tiles=None)
# Base layers
folium.TileLayer("OpenStreetMap", name="OSM", control=False).add_to(m)
folium.TileLayer("CartoDB positron", name="Light").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Dark").add_to(m)

# Cluster
cluster = MarkerCluster(name="Properties", show=True).add_to(m)

# Markers
for _, row in valid.iterrows():
    lat, lon = float(row["_lat"]), float(row["_lon"])
    popup_html = make_popup_html(row)
    tooltip = str(row.get("Property (name or address)", "")) if not pd.isna(row.get("Property (name or address)", "")) else "Property"
    folium.Marker(
        location=(lat, lon),
        tooltip=tooltip,
        popup=folium.Popup(popup_html, max_width=350),
        icon=folium.Icon(icon="info-sign")
    ).add_to(cluster)

# Optional: add a layer control
folium.LayerControl(collapsed=False).add_to(m)

# -----------------------------
# 4) Save or display
# -----------------------------
m.save("index.html")
print(f"Map saved to index.html. Skipped rows without usable coordinates: {skipped}")

m

Map saved to index.html. Skipped rows without usable coordinates: 0


In [5]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFlaSitesHittingMarket_08122025
